# 第9章 数据聚合与分组运算

# 主要内容
# 2.数据聚合
2.1 数据聚合  
2.2 面向列的多函数应用

在统计数据的时候，我们通常需要计算分组计算变量的均值、方差、最大值、最小值等等。  
所谓的数据聚合是指任何能够从数组产生标量值的数据转换过程，比如mean()返回平均值。那么在GroupBy对象中调用mean()究竟发生了什么呢？除了mean()、sum()等常见的方法，我们是否可以自己定义聚合运算呢？是否对不同的列运用不同的方法呢？能够一次对同一个列运用不同的方法呢？  

答案是可以的。

# 2.1 数据聚合

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-0.018989,-1.496759,a,one
1,0.567019,0.363657,a,two
2,-0.192725,0.247711,b,one
3,-0.835761,-2.627254,b,two
4,0.365844,0.423110,a,one


In [2]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.526784
b   -0.257028
Name: data1, dtype: float64

上述实现过程中，GroupBy高效地对Series进行切片，然后对各片调用quantile(0.9),最终将这些结果组装成最终结果。

# 自定义聚合运算

如果想要使用自己定义的聚合函数，应该怎么操作呢？  
只需将其传入aggregate或agg方法即可。  

比如，定义返回最大值和最小值之间的差值。

In [6]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.586009,1.919868
b,0.643037,2.874965


值得注意的是，自定义的聚合函数要比经过下表中经过优化的函数慢得多，这是因为构造中间分组数据块时存在非常大的开销。如函数调用、数据重排等。  

经过优化的groupby方法如下表所示：

In [8]:
from IPython.display import Image
Image(url = '3.png', width = 400, height = 300)

根据上面的过程我们可以看到，对于Series和DataFrame列的聚合运算其实就是使用aggregate或者调用mean、std等方法。  

然而，我们可能希望对不同的列使用不同的聚合函数，或者一次应用多个函数，这也是可以实现的。

# 2.2 面向列的多函数应用
- 对同一列应用不同的函数
- 对全部列应用多个函数
- 对不同的列应用不同的函数

# 对同一列应用不同的函数

这里使用一个餐馆小费的数据集进行说明

In [10]:
tips = pd.read_csv('data/tips.csv')

In [11]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [16]:
# 计算小费占总额的比例
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [13]:
# 根据sex和smoker进行分组
grouped = tips.groupby(['sex', 'smoker'])

In [17]:
grouped_pct = grouped['tip_pct']
# 可以将函数名以字符串的形式传入
grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

也可以传入一组函数

In [18]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

通过传入(name, function)元组组成的列表，那么元组的第一个元素会被作为列名

In [19]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

# 对全部列应用多个函数

假设我们想要对tip_pct和total_bill列计算三个统计信息

In [20]:
functions = ['count', 'mean', 'max']

In [21]:
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

也可以传入带有自定义名称的元组列表

In [22]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.156921   0.001327    18.105185  53.092422
       Yes        0.182150   0.005126    17.977879  84.451517
Male   No         0.160669   0.001751    19.791237  76.152961
       Yes        0.152771   0.008206    22.284500  98.244673

# 对不同的列应用不同的函数

具体方法是向agg传入一个从列名映射到函数的字典

In [27]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [28]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                    min       max      mean       std  sum
sex    smoker                                             
Female No      0.056797  0.252672  0.156921  0.036421  140
       Yes     0.056433  0.416667  0.182150  0.071595   74
Male   No      0.071804  0.291990  0.160669  0.041849  263
       Yes     0.035638  0.710345  0.152771  0.090588  150

到目前位置，所有示例中的聚合函数都有由唯一的分组键组成的索引，但是有些情况下我们并不需要这样，我们可以向groupby传入as_index=False禁用该功能

In [29]:
tips.groupby(['sex', 'smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771
